# bytes, str, unicode의 차이점을 알자

|Python3|Python2|
|:-|:-|
|- bytes와 str 두 가지 타입으로 문자 시퀀스 표현|- str과 unicode 두 가지 타입으로 문자 시퀀스 표현|
|- bytes instance는 raw 8bit 값 저장|- str instance는 raw 8bit 값 저장|
|- str instance는 unicode 문자 저장|- unicode instance는 unicode 문자 저장|

## Binary data(raw 8bit) 표현 방법

- unicode 문자를 binary data(raw 8bit)로 표현하는 방법은 많음 (대표적으로 UTF-8)
- Python3 str instance와 Python2 unicode instance는 binary data로 변환하기 위해 encode method 사용
- binary data를 unicode 문자로 변환하기 위해 decode method 사용
- Python programming 시 외부에 제공할 interface에서 unicode를 encode, decode 해야 함
- Program 핵심 부분에서 unicode type을 사용하고, 문자 encoding에 대해서ㅡㄴ 어떤 가정도 하지 말아야 함